<a href="https://colab.research.google.com/github/Chidiebere-Ogbuchi/TSP_CausalDiscovery/blob/main/Preproc_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
import pandas as pd
import numpy as np
import os
import gc
import glob

gc.collect()


0

In [176]:
## Specify file name, bandwidth and Visitors
visitor_A = 9
visitor_B = 3
visitor_C = 6

bw = 13

path = 'results_loc2_bw13.csv'

In [177]:

file_name = os.path.basename(path)  # Extract the filename from the path
file_name_without_extension, extension = os.path.splitext(file_name)  # Split filename and extension

# Add prefix or suffix to the filename
save_path = os.path.join(os.path.dirname(path), file_name_without_extension + '_pr' + extension)

print(save_path)


results_loc2_bw13_pr.csv


In [178]:
# Define column names
columns = ["timestamp", "sensorid", "observation", "location", "bandwidth", "payload", "responsetime"]

# Read the CSV file into a DataFrame with specified column names
df = pd.read_csv(path, names=columns)

# df.head()


,timestamp,sensorid,observation,location,bandwidth,payload,responsetime
0,1710853637834,loc2-vs1,loc2/temperature,LOC2,10.0,26,659
1,1710853637808,loc2-vs2,loc2/rfid,LOC2,10.0,loc2,623
2,1710853638063,loc2-vs1,loc2/temperature,LOC2,10.0,24,880
3,1710853638063,loc2-vs1,loc2/temperature,LOC2,10.0,19,880
4,1710853638085,loc2-vs1,loc2/temperature,LOC2,10.0,23,890


In [179]:
# Convert the timestamp column to a pandas datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# # Extract day, week, month, and year into separate columns
# df['day'] = df['timestamp'].dt.day
# df['week'] = df['timestamp'].dt.isocalendar().week
# df['month'] = df['timestamp'].dt.month
# df['year'] = df['timestamp'].dt.year

In [180]:
df.head()

,timestamp,sensorid,observation,location,bandwidth,payload,responsetime
0,2024-03-19 13:07:17.834,loc2-vs1,loc2/temperature,LOC2,10.0,26,659
1,2024-03-19 13:07:17.808,loc2-vs2,loc2/rfid,LOC2,10.0,loc2,623
2,2024-03-19 13:07:18.063,loc2-vs1,loc2/temperature,LOC2,10.0,24,880
3,2024-03-19 13:07:18.063,loc2-vs1,loc2/temperature,LOC2,10.0,19,880
4,2024-03-19 13:07:18.085,loc2-vs1,loc2/temperature,LOC2,10.0,23,890


In [181]:

# Change the value of 'bandwidth' column to 6
df['bandwidth'] = bw


In [182]:
import pandas as pd

# Assuming df is your DataFrame

# Convert 'timestamp' column to datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'timestamp'
df.sort_values(by='timestamp', inplace=True)

# Calculate the time range
time_range = df['timestamp'].max() - df['timestamp'].min()

# Define time intervals
interval_duration = pd.Timedelta(seconds=30)

# Define start time for each interval
start_time_A = df['timestamp'].min()
start_time_B = start_time_A + interval_duration
start_time_C = start_time_B + interval_duration

# Assign values based on time intervals
df['visitors'] = visitor_C  # Default value C
df.loc[(df['timestamp'] >= start_time_A) & (df['timestamp'] < start_time_B), 'visitors'] = visitor_A
df.loc[(df['timestamp'] >= start_time_B) & (df['timestamp'] < start_time_C), 'visitors'] = visitor_B

# print(df)


                    timestamp   sensorid         observation location  \
1     2024-03-19 13:07:17.808   loc2-vs2           loc2/rfid     LOC2   
0     2024-03-19 13:07:17.834   loc2-vs1    loc2/temperature     LOC2   
2     2024-03-19 13:07:18.063   loc2-vs1    loc2/temperature     LOC2   
3     2024-03-19 13:07:18.063   loc2-vs1    loc2/temperature     LOC2   
4     2024-03-19 13:07:18.085   loc2-vs1    loc2/temperature     LOC2   
...                       ...        ...                 ...      ...   
46233 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46234 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46235 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46236 2024-03-19 13:08:09.428  loc2-app1  loc2/comfort_level     LOC2   
46237 2024-03-19 13:08:09.428  loc2-app1  loc2/comfort_level     LOC2   

       bandwidth                   payload  responsetime  visitors  
1             13                      loc2           6

In [183]:
# Define a function to parse and assign values
def assign_value(observation):
    split = observation.split('/')
    last_part = split[-1]
    if last_part == 'rfid':
        return 1
    elif last_part == 'temperature':
        return 2
    elif last_part == 'comfort_level':
        return 3
    elif last_part == 'motion':
        return 4
    elif last_part == 'visitor_flow':
        return 5
    else:
        return None  # Return None for cases not specified

# Apply the function to the 'observation' column
df['observation_value'] = df['observation'].apply(assign_value)



In [184]:
# Define a function to map location values
def map_location(location):
    if location == 'LOC2':
        return 2
    elif location == 'LOC1':
        return 1
    else:
        return None  # Return None for other cases

# Apply the function to the 'location' column
df['location_encoded'] = df['location'].apply(map_location)

print(df)


                    timestamp   sensorid         observation location  \
1     2024-03-19 13:07:17.808   loc2-vs2           loc2/rfid     LOC2   
0     2024-03-19 13:07:17.834   loc2-vs1    loc2/temperature     LOC2   
2     2024-03-19 13:07:18.063   loc2-vs1    loc2/temperature     LOC2   
3     2024-03-19 13:07:18.063   loc2-vs1    loc2/temperature     LOC2   
4     2024-03-19 13:07:18.085   loc2-vs1    loc2/temperature     LOC2   
...                       ...        ...                 ...      ...   
46233 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46234 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46235 2024-03-19 13:08:09.427  loc2-app1  loc2/comfort_level     LOC2   
46236 2024-03-19 13:08:09.428  loc2-app1  loc2/comfort_level     LOC2   
46237 2024-03-19 13:08:09.428  loc2-app1  loc2/comfort_level     LOC2   

       bandwidth                   payload  responsetime  visitors  \
1             13                      loc2           

In [185]:
columns_to_keep = ["bandwidth", "responsetime", "visitors", "observation_value", 'location_encoded']
df_selected = df[columns_to_keep]

print(df_selected)

       bandwidth  responsetime  visitors  observation_value  location_encoded
1             13           623         9                  1                 2
0             13           659         9                  2                 2
2             13           880         9                  2                 2
3             13           880         9                  2                 2
4             13           890         9                  2                 2
...          ...           ...       ...                ...               ...
46233         13         15424         3                  3                 2
46234         13         15395         3                  3                 2
46235         13         15395         3                  3                 2
46236         13         15396         3                  3                 2
46237         13         15391         3                  3                 2

[46238 rows x 5 columns]


In [186]:
# Save the DataFrame to a CSV file
df_selected.to_csv(save_path, index=False)

# Optionally, you can specify index=False if you don't want to save the index as a separate column in the CSV file.


### Consider using below to combine after initial pre-processing above

In [194]:
# Use glob to get a list of files ending with '_output.csv' in the root directory
output_files = glob.glob('*_pr.csv')

# Check if there are any files to concatenate
if output_files:
    # Read each CSV file and concatenate into a single DataFrame
    combined_df = pd.concat([pd.read_csv(file) for file in output_files], ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    # Create the directory if it doesn't exist
    output_directory = './data/'
    os.makedirs(output_directory, exist_ok=True)

    combined_csv_path = './data/output.csv'
    combined_df.to_csv(combined_csv_path, index=False)

    print(f'Combined CSV file saved at: {combined_csv_path}')
else:
    print('No files ending with "_output.csv" found in the root directory.')


Combined CSV file saved at: ./data/output_full_pr.csv
